In [99]:
#import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

<h1>Pobieranie danych z bazy</h1>

In [100]:
data = {}

dbtables = ['ONI_CIRCUITS', 'MEB_CONTAINERS', 'MEB_DGM', 'MEB_DMC', 'MEB_DMC_GRADE', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'MEB_KS_WZORC']
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [101]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table in dbtables:
        query = f"select * from z3dmc.{table} where rownum<101"
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

<h1>Łączenie tabel</h1>

In [102]:
data['MEB_DGM'] = data['MEB_DGM'].merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
data['MEB_DGM']

,id,dmc,nr_dgm,status,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,...,id_dmc,nok_strefa,nok_rodzaj,status_ko,data_odlania_y,timestamp_y,operator,rodzaj_kontroli,product_id,line_id
0,220,190627091611011003610,10,4,1850,88,47.5,6.52,7,264,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
1,3416,190719071611011006077,10,1,2197,89,46.5,6.59,11,285,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
2,3417,190719071811011006078,10,1,2184,86,45.0,6.52,8,284,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
3,3418,190719072011011006079,10,1,2159,89,45.5,6.54,6,284,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
4,3419,190719072111011006080,10,1,2175,89,47.0,6.57,8,283,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3523,190719104911011006186,10,1,2134,88,45.5,6.51,14,278,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
96,2802,190715085311011005398,10,1,1939,88,46.0,6.54,5,290,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
97,318,190627091611011003611,10,4,1851,87,48.0,6.63,9,265,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
98,319,190627091611011003612,10,4,1864,85,48.0,6.57,8,266,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN


In [103]:
oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'ref_time', 'set_point', 'start_delay', 'temp', 'working_mode'])
oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
oni_circuits.reset_index(inplace=True)

oni_circuits = oni_circuits.merge(data['MEB_DGM'], on='id_dmc', how='left')
for x in oni_circuits.columns[216:]: oni_circuits.pop(x)
oni_circuits.head()

,id_dmc,assigment_1,assigment_2,assigment_3,assigment_4,assigment_6,assigment_7,assigment_8,assigment_9,assigment_10,...,id_ko,nok_strefa,nok_rodzaj,status_ko,data_odlania_y,timestamp_y,operator,rodzaj_kontroli,product_id,line_id
0,319248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
1,319320,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
2,319339,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
3,319340,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
4,319399,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN


In [104]:
data['MEB_DMC'] = data['MEB_DMC'].merge(oni_circuits, on='id_dmc', how='left')
data['MEB_DMC'].pop('timestamp')
data['MEB_DMC']


,id_dmc,dmc_x,status_koncowy,update_time,id_meb_containers,packed_time,first_packed_time,dmc_casting,production_step,assigment_1,...,id_ko,nok_strefa,nok_rodzaj,status_ko,data_odlania_y,timestamp_y,operator,rodzaj_kontroli,product_id,line_id
0,2063,0MH301103CVW41910090436321102S111910040455,2,2019-11-06 05:40:27,366,2019-11-06 05:40:26,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
1,2064,0MH301103CVW41910090436411102S111910040410,1,2019-11-06 05:50:29,0,2019-10-09 08:18:06,None,None,500.0,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
2,2065,0MH301103CVW41910090436521102S111910040350,2,2019-11-06 06:47:19,366,2019-11-06 06:47:18,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
3,2066,0MH301103CVW41910090436611102S111910040356,2,2019-11-06 05:40:39,366,2019-11-06 05:40:38,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
4,2067,0MH301103CVW41910090436721102S111910040345,2,2019-10-09 08:22:31,0,NaT,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2544,0MH301103CVW41909060331111202S111909030756,2,2019-11-05 08:11:34,366,2019-11-05 08:11:34,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
96,2545,0MH301103CVW41909060331621202S111909030732,1,2019-11-05 06:46:44,488,2019-11-05 06:46:43,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
97,2546,0MH301103CVW41909060330821202S111909030737,2,2019-11-05 08:17:28,366,2019-11-05 08:17:28,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
98,2547,0MH301103CVW41910100485511102S111910031931,1,2019-11-06 00:22:38,498,2019-11-06 00:22:38,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN


In [105]:
data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_GROB'], on='id_dmc', how='left')
data['MEB_DMC']

,id_dmc,dmc_x,status_koncowy,update_time,id_meb_containers,packed_time,first_packed_time,dmc_casting,production_step,assigment_1,...,workingstep1,workingstep2,workingstep3,workingstep4,mms_ok,reworkrequested,reworkdone,partcleaningisfinished,waitfortoolcheck,machine_nr
0,2063,0MH301103CVW41910090436321102S111910040455,2,2019-11-06 05:40:27,366,2019-11-06 05:40:26,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2064,0MH301103CVW41910090436411102S111910040410,1,2019-11-06 05:50:29,0,2019-10-09 08:18:06,None,None,500.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2065,0MH301103CVW41910090436521102S111910040350,2,2019-11-06 06:47:19,366,2019-11-06 06:47:18,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2066,0MH301103CVW41910090436611102S111910040356,2,2019-11-06 05:40:39,366,2019-11-06 05:40:38,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2067,0MH301103CVW41910090436721102S111910040345,2,2019-10-09 08:22:31,0,NaT,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2544,0MH301103CVW41909060331111202S111909030756,2,2019-11-05 08:11:34,366,2019-11-05 08:11:34,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,2545,0MH301103CVW41909060331621202S111909030732,1,2019-11-05 06:46:44,488,2019-11-05 06:46:43,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,2546,0MH301103CVW41909060330821202S111909030737,2,2019-11-05 08:17:28,366,2019-11-05 08:17:28,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,2547,0MH301103CVW41910100485511102S111910031931,1,2019-11-06 00:22:38,498,2019-11-06 00:22:38,None,None,700.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
"""connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')
connection.version
connection.close()"""

"connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')\nconnection.version\nconnection.close()"

In [107]:
"""try:
    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(dmc_sql)
            while True:
                row = cursor.fetchone()
                if row is None:
                    break
                print(row)
                data={'id':row[0],'dmc':row[1]}
    cursor.close()
    connection.close()
except cx_Oracle.Error as error:
    print(error)"""

"try:\n    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:\n        with connection.cursor() as cursor:\n            cursor.execute(dmc_sql)\n            while True:\n                row = cursor.fetchone()\n                if row is None:\n                    break\n                print(row)\n                data={'id':row[0],'dmc':row[1]}\n    cursor.close()\n    connection.close()\nexcept cx_Oracle.Error as error:\n    print(error)"

In [108]:
#dodej to Adam przystojniaku